In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import csv
import yaml

In [2]:
configFile = '../pipeline/config.yml'

In [3]:
# Constants
SPARQL = 0
CSV = 1

In [4]:
try:
    with open(configFile, 'r') as f:
        config = yaml.safe_load(f)
except:
    raise Exception("Could not load config file at", configFile)

### Define functions

In [5]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in results["head"]["vars"]:
            if key in result:
                row[key] = result[key]["value"]
            else:
                row[key] = None
        rows.append(row)
    return rows

## Step 1: Get input data

In [6]:
mode = False
if config['mode'] == "SPARQL":
    mode = SPARQL
elif config['mode'] == "CSV":
    mode  = CSV
else:
    raise Exception("mode not specified or invalid (should be SPARQL or CSV)")

Read data from input file, if present. This is being done for both CSV and SPARQL mode as the SPARQL results will be cashed in the CSV file and updated when data is changed.

In [7]:
inputData = []
try:
    with open(config['dataFile'], 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            inputData.append({
                "id": row['id'],
                "image": row['image'],
                "width": row['width'],
                "height": row['height'],
            })
except:
    print("No prior input file found")


If in SPARQL mode, get data from SPARQL endpoint

In [8]:
if mode == SPARQL:
    if not config['endpoint'] or not config['query']:
        raise Exception("incomplete configuration for SPARQL mode")
        
    sparql = SPARQLWrapper(config['endpoint'], returnFormat=JSON)
    sparql.setQuery(config['query'])
    try:
        ret = sparql.query().convert()
    except:
        raise Exception("Could not execute query against endpoint", config['endpoint'])
    queriedData = sparqlResultToDict(ret)

If in SPARQL mode, merge queried data with data stored in CSV file.
- add entries that exist in SPARQL result, but not in the CSV file
- add width/height information when it is only available in either the CSV file or the SPARQL output (prioritising the SPARQL data)
Store merged data in CSV file

In [9]:
if mode == SPARQL:
    inputDataHash = {}
    queriedDataHash = {}

    for row in inputData:
        inputDataHash[row['id']] = row
    for row in queriedData:
        queriedDataHash[row['id']] = row

    data = inputData
    idsInInputData = [d['id'] for d in inputData]
    for row in queriedData:
        if row['id'] not in idsInInputData:
            data.append(row)

    for row in data:
        if not row['width']:
            if row['id'] in queriedDataHash and queriedDataHash[row['id']]['width']:
                row['width'] = queriedDataHash[row['id']]['width']
            elif row['id'] in inputDataHash and inputDataHash[row['id']]['width']:
                row['width'] = inputDataHash[row['id']]['width']
        if not row['height']:
            if row['id'] in queriedDataHash and queriedDataHash[row['id']]['height']:
                row['height'] = queriedDataHash[row['id']]['height']
            elif row['id'] in inputDataHash and inputDataHash[row['id']]['width']:
                row['height'] = inputDataHash[row['id']]['height']
                
    try:
        with open(config['dataFile'], 'w') as f:
            writer = csv.DictWriter(f, fieldnames=['id','image','width','height'])
            writer.writeheader()
            for row in data:
                writer.writerow(row)
    except:
        raise Exception("Could not write to", config['dataFile'])
else:
    data = 


SyntaxError: invalid syntax (<ipython-input-9-6fa970eada8e>, line 1)